自動轉換為簡體中文

In [ ]:
import yt_dlp
import requests
from bs4 import BeautifulSoup
import re
import opencc

# 使用者輸入影片名稱和集數區間
video_title_traditional = input("請輸入影片名稱: ")
start_episode = int(input("請輸入開始集數: "))
end_episode = int(input("請輸入結束集數: "))

# 將繁體中文轉為簡體中文
converter = opencc.OpenCC('t2s')
video_title = converter.convert(video_title_traditional)

# 官網首頁 URL
base_url = "https://www.renren.pro/"

# 設定 yt-dlp 的選項
ydl_opts = {
    # 選擇最高解析度，並確保包含影音
    'format': 'bestvideo+bestaudio/best',
    # 合併為 mp4 格式
    'merge_output_format': 'mp4',
    # 如果是播放清單只下載單個影片
    'noplaylist': True
}

def search_video_url(video_title):
    # 搜索影片，正確設置參數名稱 wd
    search_url = f"{base_url}search?wd={video_title}"
    response = requests.get(search_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 搜尋結果中找到影片主頁 URL（假設搜尋結果包含在 <a> 標籤內）
        for link in soup.find_all('a', href=True):
            href = link['href']
            if '/play/' in href:
                # 返回影片主頁的完整 URL
                return base_url.strip("/") + href

    print("無法找到影片主頁，請檢查影片名稱。")
    return None

def extract_episode_urls(video_main_url):
    response = requests.get(video_main_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        episode_urls = []

        # 解析網頁中包含的集數 URL（假設在 <a> 標籤中或有相似結構）
        for link in soup.find_all('a', href=True):
            href = link['href']
            if '/play/' in href:
                full_url = base_url.strip("/") + href
                episode_urls.append(full_url)

        # 返回所有集數的 URL 列表
        return episode_urls
    else:
        print("無法抓取影片主頁內容。")
        return []

def extract_m3u8_url(page_content):
    # 使用正則表達式尋找 .m3u8 連結
    m3u8_regex = re.compile(r'https?://[^\s\'"]+\.m3u8')
    m3u8_url = m3u8_regex.search(page_content)
    if m3u8_url:
        return m3u8_url.group(0)
    return None

def download_video(video_url, episode_number):
    output_filename = \
        f"{video_title_traditional}_"\
        f"第{episode_number}集.%(ext)s"
    try:
        # 嘗試直接下載影片，並指定集數為文件名稱
        with yt_dlp.YoutubeDL({
            **ydl_opts,
            'outtmpl': output_filename
        }) as ydl:
            ydl.download([video_url])
    except yt_dlp.utils.DownloadError:
        print(
            f"第 {episode_number} 集直接下載失敗，"
            "嘗試提取 .m3u8 串流鏈接..."
        )

        # 使用 requests 和 BeautifulSoup 抓取網頁內容
        response = requests.get(video_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            m3u8_url = None

            # 嘗試從 script 標籤中提取 .m3u8 連結
            for script in soup.find_all('script'):
                m3u8_url = extract_m3u8_url(script.text)
                if m3u8_url:
                    break

            if m3u8_url:
                print(
                    f"第 {episode_number} 集找到 "
                    f".m3u8 串流鏈接: {m3u8_url}"
                )
                try:
                    # 使用 m3u8 串流鏈接下載影片
                    with yt_dlp.YoutubeDL({
                        **ydl_opts,
                        'outtmpl': output_filename
                    }) as ydl:
                        ydl.download([m3u8_url])
                except yt_dlp.utils.DownloadError:
                    print(
                        f"第 {episode_number} "
                        "集無法下載 .m3u8 串流鏈接。"
                    )
            else:
                print(
                    f"第 {episode_number} "
                    "集未找到有效的 .m3u8 串流鏈接。"
                )
        else:
            print(f"第 {episode_number} 集無法抓取網頁內容。")

# 步驟 1：搜索並獲取影片主頁 URL
video_main_url = search_video_url(video_title)
if not video_main_url:
    # 若未找到影片主頁，則退出程式
    exit()

# 步驟 2：解析主頁，獲取所有集數的 URL
episode_urls = extract_episode_urls(video_main_url)

# 步驟 3：下載範圍內的每一集
for episode_number in range(
    start_episode,
    min(end_episode + 1, len(episode_urls) + 1)
):
    # 調整索引以匹配集數
    episode_url = episode_urls[episode_number - 1]
    print(
        f"下載 {video_title_traditional} "
        f"第 {episode_number} 集: {episode_url}"
    )
    download_video(episode_url, episode_number)
